# HOMEWORK 11
Full name: Nguyễn Thị Minh Ngọc
<br>
Student ID: 11219280
<br>
Class: DSEB 63

In [1]:
from tabulate import tabulate

# Problem 1:  Separate Chaining method

### a. Implementation

In [2]:
class Node:
    def __init__(self, data, nextnode = None):
        self.data = data
        self.next = nextnode
        
    def __repr__(self):
        return f'{self.data[0]}: {self.data[1]}'
    
class SepChainHash:
    def __init__(self, size):
        self.size = size
        self.map = [None] * self.size
        
    def _hash(self, key):
        if isinstance(key, bool):
            key = str(key)
        if isinstance(key, str):
            key = [ord(i) for i in key]
            key = sum(key)
        return round(key%self.size)
    
    def _convert_key(self, key):
        '''Goals:
        - Distinguish between string and integer and bool (For example, key '6' is different from 6, True if different
        from 'True')
        - Convert boolen key into string form'''
        if isinstance(key, str):
            return f"'{str(key)}'"
        elif isinstance(key, bool):
            return str(key)
        return key

    def add(self, key, value):
        # Update the value if there are duplicate keys
        index = self._hash(key)
        key = self._convert_key(key)
        node = self.map[index]
        if not node:
            self.map[index] = Node([key, value])
        else:
            while node:
                if node.data[0] == key:
                    node.data[1] = value
                    break
                if not node.next:
                    node.next = Node([key, value], node.next)
                node = node.next
                    
    def __getitem__(self, key):
        index = self._hash(key)
        key = self._convert_key(key)
        node = self.map[index]
        while node:
            if node.data[0] == key:
                return node.data[1]
            node = node.next
        return None
    
    def delete(self, key):
        index = self._hash(key)
        key = self._convert_key(key)
        node = self.map[index]
        if not node:
            raise KeyError("Key can't be found")
        else:
            if node.data[0] == key:
                self.map[index] = node.next
            else:
                while node.next and node.next.data[0] != key:
                    node = node.next
                if node.next:
                    node.next = node.next.next
                else:
                    raise KeyError("Key can't be found")
    
    def display(self):
        output = []
        for i in range(len(self.map)):
            temp = []
            temp.append(i)
            temp.append(self.map[i])
            node = self.map[i]
            while node and node.next:
                node = node.next
                temp.append(node)
            output.append(temp)
        print(tabulate(output, tablefmt='fancy_grid'))

    def __repr__(self):
        output = '{'
        for i in range(len(self.map)):
            node = self.map[i]
            while node:
                output += f'{node}, '
                node = node.next
        output = output[:-2:]
        output += '}'
        return output

### b. Check your implementation

In [3]:
# Create a SepChainHash object with size 5
data = [(6, 'a'), (1, 'b'), (12, 'h'), (10, 'r'), (6, 'p'), (4, 's'), (2, 'n')]
testcase = SepChainHash(5)
for i in data:
    testcase.add(i[0], i[1])
print(testcase)

{10: r, 6: p, 1: b, 12: h, 2: n, 4: s}


In [4]:
testcase.display()

╒═══╤═══════╤══════╕
│ 0 │ 10: r │      │
├───┼───────┼──────┤
│ 1 │ 6: p  │ 1: b │
├───┼───────┼──────┤
│ 2 │ 12: h │ 2: n │
├───┼───────┼──────┤
│ 3 │       │      │
├───┼───────┼──────┤
│ 4 │ 4: s  │      │
╘═══╧═══════╧══════╛


In [5]:
#  Get the items with key 9
print('The item with key 9:', testcase[9])

The item with key 9: None


In [6]:
 # Get the items with key 4
print('The item with key 4:', testcase[4])

The item with key 4: s


In [7]:
# Delete the item with key 6
testcase.delete(6)
print(testcase)

{10: r, 1: b, 12: h, 2: n, 4: s}


In [8]:
testcase.display()

╒═══╤═══════╤══════╕
│ 0 │ 10: r │      │
├───┼───────┼──────┤
│ 1 │ 1: b  │      │
├───┼───────┼──────┤
│ 2 │ 12: h │ 2: n │
├───┼───────┼──────┤
│ 3 │       │      │
├───┼───────┼──────┤
│ 4 │ 4: s  │      │
╘═══╧═══════╧══════╛


# Problem 2:  Linear Probing method

In [9]:
class Full(Exception):
    pass

### a. Implementation

In [10]:
class LinProbHash:
    def __init__(self, size):
        self.size = size
        self.map = [None] * size
    
    def _hash(self, key, update=0):
        if isinstance(key, bool):
            key = str(key)
        if isinstance(key, str):
            key = [ord(i) for i in key]
            key = sum(key)
        key += update
        return round(key%self.size)
    
    def _convert_key(self, key):
        '''Goals:
        - Distinguish between string and integer and bool (For example, key '6' is different from 6)
        - Convert boolen key into string form'''
        if isinstance(key, str):
            return f"'{str(key)}'"
        elif isinstance(key, bool):
            return str(key)
        return key
    
    def add(self, key, value):
        # Update the value if there are duplicate keys
        index = self._hash(key)
        key = self._convert_key(key)
        update = 0
        while self.map[index] and self.map[index][0] != key:
            update += 1
            index = self._hash(key, update)
            if update >= self.size:
                raise Full('Map is full')
        self.map[index] = (key, value)
    
    def __getitem__(self, key):
        index = self._hash(key)
        key = self._convert_key(key)
        update = 0
        while update < self.size:
            if self.map[index] and self.map[index][0] == key:
                return f'{self.map[index][1]}'
            update += 1
            index = self._hash(key, update)
        return None        
    
    def delete(self, key):
        index = self._hash(key)
        key = self._convert_key(key)
        check = False
        update = 0
        while update < self.size and self.map[index]:
            if self.map[index][0] == key:
                check = True
                break
            update += 1
            index = self._hash(key, update)
        if check:
            self.map[index] = None
        else:
            raise KeyError("Key can't be found")
    
    def display(self):
        index = []
        data = []
        for i in range(len(self.map)):
            index.append(i)
            if self.map[i]:
                data.append(f'{self.map[i][0]}: {self.map[i][1]}')
            else:
                data.append(self.map[i])
        output = [index, data]
        print(tabulate(output, tablefmt='fancy_grid'))
    
    def __repr__(self):
        output = '{'
        for i in range(len(self.map)):
            if self.map[i]:
                output += f'{self.map[i][0]}: {self.map[i][1]}, '
        output = output[:-2:]
        output += '}'
        return output

### b. Check your implementation

In [11]:
# Create a LinProbHash object with size 8
data2 = [(6, 's'), (3, 'd'), (11, 'a'), (19, 'p'), (8, 'd'), (14, 'q'), (21, 'j')]
testcase2 = LinProbHash(8)
for i in data2:
    testcase2.add(i[0], i[1])
print(testcase2)

{8: d, 21: j, 3: d, 11: a, 19: p, 6: s, 14: q}


In [12]:
testcase2.display()

╒══════╤═══════╤═══╤══════╤═══════╤═══════╤══════╤═══════╕
│ 0    │ 1     │ 2 │ 3    │ 4     │ 5     │ 6    │ 7     │
├──────┼───────┼───┼──────┼───────┼───────┼──────┼───────┤
│ 8: d │ 21: j │   │ 3: d │ 11: a │ 19: p │ 6: s │ 14: q │
╘══════╧═══════╧═══╧══════╧═══════╧═══════╧══════╧═══════╛


In [13]:
# Delete the items with key 11
testcase2.delete(11)
print('testcase2.delete(11)  >>>  ', str(testcase2))

# Delete the items with key 3
testcase2.delete(3)
print('testcase2.delete(3)   >>>  ',str(testcase2))

testcase2.delete(11)  >>>   {8: d, 21: j, 3: d, 19: p, 6: s, 14: q}
testcase2.delete(3)   >>>   {8: d, 21: j, 19: p, 6: s, 14: q}


In [14]:
testcase2.display()

╒══════╤═══════╤═══╤═══╤═══╤═══════╤══════╤═══════╕
│ 0    │ 1     │ 2 │ 3 │ 4 │ 5     │ 6    │ 7     │
├──────┼───────┼───┼───┼───┼───────┼──────┼───────┤
│ 8: d │ 21: j │   │   │   │ 19: p │ 6: s │ 14: q │
╘══════╧═══════╧═══╧═══╧═══╧═══════╧══════╧═══════╛


In [15]:
# Add (16 , 'l')
testcase2.add(16, 'l')
print("testcase2.add(16, 'l')\t>>>  ", str(testcase2))

# Add (0 , 'o')
testcase2.add(0, 'o')
print("testcase2.add(0, 'o')\t>>>  ", str(testcase2))

# Add (9 , 'a')
testcase2.add(9, 'a')
print("testcase2.add(9, 'a')\t>>>  ", str(testcase2))

testcase2.add(16, 'l')	>>>   {8: d, 21: j, 16: l, 19: p, 6: s, 14: q}
testcase2.add(0, 'o')	>>>   {8: d, 21: j, 16: l, 0: o, 19: p, 6: s, 14: q}
testcase2.add(9, 'a')	>>>   {8: d, 21: j, 16: l, 0: o, 9: a, 19: p, 6: s, 14: q}


In [16]:
testcase2.display()

╒══════╤═══════╤═══════╤══════╤══════╤═══════╤══════╤═══════╕
│ 0    │ 1     │ 2     │ 3    │ 4    │ 5     │ 6    │ 7     │
├──────┼───────┼───────┼──────┼──────┼───────┼──────┼───────┤
│ 8: d │ 21: j │ 16: l │ 0: o │ 9: a │ 19: p │ 6: s │ 14: q │
╘══════╧═══════╧═══════╧══════╧══════╧═══════╧══════╧═══════╛


In [17]:
#  Delete item with key 8
testcase2.delete(8)
print('testcase2.delete(8)  >>>  ', str(testcase2))

testcase2.delete(8)  >>>   {21: j, 16: l, 0: o, 9: a, 19: p, 6: s, 14: q}


In [18]:
testcase2.display()

╒═══╤═══════╤═══════╤══════╤══════╤═══════╤══════╤═══════╕
│ 0 │ 1     │ 2     │ 3    │ 4    │ 5     │ 6    │ 7     │
├───┼───────┼───────┼──────┼──────┼───────┼──────┼───────┤
│   │ 21: j │ 16: l │ 0: o │ 9: a │ 19: p │ 6: s │ 14: q │
╘═══╧═══════╧═══════╧══════╧══════╧═══════╧══════╧═══════╛


In [19]:
testcase2[14]

'q'

# SECOND TIME

# Problem 1

In [20]:
class Node:
    def __init__(self, value, nextnode=None):
        self.value = value
        self.next = nextnode
    
    def __repr__(self):
        return str(self.value)

class SepChainHash:
    def __init__(self, size):
        pass
    
    def _hash(self, key):
        pass
    
    def add(self, node):
        pass
    
    def __getitem__(self):
        pass
    
    def delelte(self, key):
        pass
    
    def __repr__(self):
        pass

In [21]:
class LinProbHash:
    def __init__(self, size=5):
        self._size = size
        self._table = [None] * size
        
    def _hash(self, key):
        if isinstance(key, bool):
            key = str(key)
        if isinstance(key, str):
            key = sum([ord(i) for i in key])
        return round(key%self._size)
    
    def add(self, key, value):
        index = self._hash(key)
        while self._table[index]:
            if self._table[index][0] == key:
                break
            index = self._hash(index+1) 
        self._table[index] = (key, value)
    
    def __getitem__(self, key):
        index = self._hash(key)
        while self._table[index]:
            if self._table[index][0] == key:
                return f'{self._table[index][0]}: {self._table[index][1]}'
            index = self._hash(index+1)
    
    def delete(self, key):
        index = self._hash(key)
        update = 0
        removed = None
        while self._table[index]:
            if self._table[index][0] == key:
                removed = self._table[index]
                self._table[index] = None
            else:
                index = self._hash(index+1)
        if not removed:
            raise KeyError("Key cannot be found")
        index += 1
        while self._table[index]:
            current = self._table[index]
            self._table[index] = None
            self.add(current[0], current[1])
            index = self._hash(index+1)
        return removed
        
    def __repr__(self):
        # return str(self._table)
        index = []
        items = []
        for i in range(len(self._table)):
            index.append(i)
            if self._table[i]:
                if isinstance(self._table[i][0], str):
                    key = f"'{self._table[i][0]}'"
                else:
                    key = self._table[i][0]
                if isinstance(self._table[i][1], str):
                    value = f"'{self._table[i][1]}'"
                else:
                    valie = self._table[i][1]
                items.append(f'{key}: {value}')
            else:
                items.append(self._table[i])
        print(tabulate([index, items], tablefmt='fancy_grid'))
        return ''

In [22]:
# Create a LinProbHash object with size 8
data3 = [(6, 's'), (3, 'd'), (11, 'a'), (19, 'p'), (8, 'd'), (14, 'q'), (21, 'j')]
testcase3 = LinProbHash(8)
for i in data3:
    testcase3.add(i[0], i[1])
print(testcase3)

╒════════╤═════════╤═══╤════════╤═════════╤═════════╤════════╤═════════╕
│ 0      │ 1       │ 2 │ 3      │ 4       │ 5       │ 6      │ 7       │
├────────┼─────────┼───┼────────┼─────────┼─────────┼────────┼─────────┤
│ 8: 'd' │ 21: 'j' │   │ 3: 'd' │ 11: 'a' │ 19: 'p' │ 6: 's' │ 14: 'q' │
╘════════╧═════════╧═══╧════════╧═════════╧═════════╧════════╧═════════╛



In [23]:
# Delete the items with key 11
testcase3.delete(11)
print('testcase3.delete(11)  >>>  ')
print(testcase3)

testcase3.delete(11)  >>>  
╒════════╤═══╤═══╤════════╤═════════╤═════════╤════════╤═════════╕
│ 0      │ 1 │ 2 │ 3      │ 4       │ 5       │ 6      │ 7       │
├────────┼───┼───┼────────┼─────────┼─────────┼────────┼─────────┤
│ 8: 'd' │   │   │ 3: 'd' │ 19: 'p' │ 21: 'j' │ 6: 's' │ 14: 'q' │
╘════════╧═══╧═══╧════════╧═════════╧═════════╧════════╧═════════╛



In [24]:
print(testcase3[8])
print(testcase3[21])
print(testcase3[3])
print(testcase3[19])
print(testcase3[6])
print(testcase3[14])

8: d
21: j
3: d
19: p
6: s
14: q


In [25]:
# Delete the items with key 3
testcase3.delete(3)
print('testcase3.delete(3)  >>>  ')
print(testcase3)

testcase3.delete(3)  >>>  
╒════════╤═══╤═══╤═════════╤═══╤═════════╤════════╤═════════╕
│ 0      │ 1 │ 2 │ 3       │ 4 │ 5       │ 6      │ 7       │
├────────┼───┼───┼─────────┼───┼─────────┼────────┼─────────┤
│ 8: 'd' │   │   │ 19: 'p' │   │ 21: 'j' │ 6: 's' │ 14: 'q' │
╘════════╧═══╧═══╧═════════╧═══╧═════════╧════════╧═════════╛

